# Schrodinger equation problems

---

Some problems about numerically solving the time-dependent Schrodinger equation following [this book](https://websites.umich.edu/~mejn/cp/index.html).

---

### 9.8: The Schrodinger equation and the Crank-Nicolson method

In one dimension the Schrodinger equation for a particle of mass $m$ with no potential energy reads

$$
- \frac{\hbar^2}{2m} \frac{\partial^2 \psi}{\partial x^2} = i \hbar \frac{\partial \psi}{\partial t}
$$

For simplicity, let's put our particle in a box with impenetrable walls, so that we only have to solve the equation in a finite-sized space. The box forces the wave function $\psi$ to be zero at the walls, which we'll put at $x=0$ and $x=L$.

Replacing the second derivative in the Schrodinger equation with a finite difference and applying Euler's method, we get the FTCS equation

$$
\psi(x, t+h) = \psi(x, t) + h \frac{i \hbar}{2ma^2}[\psi(x + a, t) + \psi(x - a, t) - 2 \psi(x, t)]
$$

where $a$ is the spacing of the spatial grid points and $h$ is the size of the time step. Be careful not to confuse the time step $h$ with Planck's constant $\hbar$. Performing a similar step in reverse, we get the implicit equation

$$
\psi(x, t+h) - h \frac{i \hbar}{2ma^2}[\psi(x + a, t+ h) + \psi(x - a, t+h) - 2 \psi(x, t+h)]= \psi(x, t) 
$$

And taking the average of these two, we get the Crank-Nicolson equation for the Schrodinger equation

\begin{align*}
\psi(x,t+h) - h {i \hbar\over4ma^2} \bigl[ &\psi(x+a,t+h)
              + \psi(x-a,t+h) - 2\psi(x,t+h) \bigr] \nonumber\\
  &= \psi(x,t) + h {i \hbar\over4ma^2} \bigl[ \psi(x+a,t)
              + \psi(x-a,t) - 2\psi(x,t) \bigr].
\end{align*}

This gives us a set of simultaneous equations, one for each grid point.

The boundary conditions on our problem tell us that $\psi=0$ at
$x=0$ and $x=L$ for all $t$.  In between these points we have grid points
at $a$, $2a$, $3a$, and so forth.  Let us arrange the values of $\psi$ at
these interior points into a vector

$$
\psi(t)
  = \begin{pmatrix} \psi(a,t) \\ \psi(2a,t) \\ \psi(3a,t) \\ \vdots
    \end{pmatrix}.
$$

Then the Crank-Nicolson equations can be written in the form

$$
A\psi(t+h) = B\psi(t),
$$

where the matrices $A$ and $B$ are both symmetric and
tridiagonal:

$$
A = \begin{pmatrix} a_1 & a_2 \\
                          a_2 & a_1 & a_2 \\
                              & a_2 & a_1 & a_2 \\
                              &     & a_2 & a_1 \\
                              &     &     &     & \ddots
          \end{pmatrix},\qquad\qquad
B = \begin{pmatrix} b_1 & b_2 \\
                          b_2 & b_1 & b_2 \\
                              & b_2 & b_1 & b_2 \\
                              &     & b_2 & b_1 \\
                              &     &     &     & \ddots
          \end{pmatrix},
$$

with

$$
a_1 = 1 + h {i\hbar\over2ma^2},\qquad
a_2 = - h {i\hbar\over4ma^2},\qquad
b_1 = 1 - h {i\hbar\over2ma^2},\qquad
b_2 = h {i\hbar\over4ma^2}.
$$

(Note the different signs and the factors of 2 and 4 in the denominators.)

The equation $A\psi(t+h) = B\psi(t)$
has precisely the form $A\vec{x} = \vec{v}$ of the simultaneous
equation problems we studied in Chapter 6 and can be solved using the same
methods.  Specifically, since the matrix $A$ is tridiagonal in this
case, we can use the fast tridiagonal version of Gaussian elimination that
we looked at in Section 6.1.6.


<div class="alert alert-info">
    <b>Tridiagnoal matrix quick solve</b> 

coming soon...
</div>



Consider an electron (mass $M=9.109\times10^{-31}\,$ kg) in a box of
length $L=10^{-8}\,$ m.  Suppose that at time $t=0$ the wavefunction of the
electron has the form

$$
\psi(x,0) = \exp \biggl[ -{(x-x_0)^2\over2\sigma^2} \biggr] e^{i\kappa x},
$$

where

$$
x_0 = {L\over2},\qquad
\sigma = 1\times10^{-10}\,\mathrm{m},\qquad
\kappa = 5\times10^{10}\,\mathrm{m}^{-1},
$$

and $\psi=0$ on the walls at $x=0$ and $x=L$.  (This expression for
$\psi(x,0)$ is not normalized---there should really be an overall
multiplying coefficient to make sure that the probability density for the
electron integrates to unity.  It's safe to drop the constant, however,
because the Schrodinger equation is linear, so the constant cancels out
on both sides of the equation and plays no part in the solution.)

Write a program to perform a single step of the
Crank-Nicolson method for this electron, calculating the vector
$\psi(t)$ of values of the wavefunction, given the initial
wavefunction above and using $N=1000$ spatial slices with $a=L/N$.  Your
program will have to perform the following steps.  First, given the
vector $\psi(0)$ at $t=0$, you will have to multiply by the
matrix $B$ to get a vector $\vec{v} = B\psi$.
Because of the tridiagonal form of $B$, this is fairly simple.  The
$i$ th component of $\vec{v}$ is given by

$$
v_i = b_1\psi_i + b_2(\psi_{i+1}+\psi_{i-1}).
$$

You will also have to choose a value for the time-step $h$.  A reasonable
choice is $h=10^{-18}\,$ s.

Second you will have to solve the linear system $Ax=\vec{v}$
for $\vec{x}$, which gives you the new value of $\psi$.  You
could do this using a standard linear equation solver like the function
solve in numpy.linalg, but since the matrix $A$ is
tridiagonal a better approach would be to use the fast solver for banded
matrices given in Appendix E, which can be imported from the file
banded.py (which you can find in the on-line resources).  This solver works fine with complex-valued arrays, which you'll need to use to represent the wavefunction $\psi$ and the matrix $A$.

Third, once you have the code in place to perform a single step of the
calculation, extend your program to perform repeated steps and hence solve
for $\psi$ at a sequence of times a separation $h$ apart.  Note that the
matrix $A$ is independent of time, so it doesn't change from one step
to another.  You can set up the matrix just once and then keep on reusing
it for every step.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

---

### 9.9: The Schrodinger equation and the spectral method

This exercise uses the spectral method to solve the time-dependent Schodinger equation

$$
-{\hbar^2\over2m} {\partial^2\psi\over\partial x^2}
  = i \hbar {\partial\psi\over\partial t}
$$

for the same system as in Exercise 9.8, a single particle in one dimension
in a box of length $L$ with impenetrable walls.  The wave function in such a
box necessarily goes to zero on the walls and hence one possible
(unnormalized) solution of the equation is

$$
\psi_k(x,t) = \sin \biggl( {\pi k x\over L} \biggr)\,e^{i Et/\hbar},
$$

where the energy~$E$ can be found by substituting into the Schrodinger equation, giving

$$
E = {\pi^2\hbar^2k^2\over2mL^2}.
$$

As with the vibrating string of Section 9.3.4, we can write a full solution
as a linear combination of such individual solutions, which on the grid
points $x_n=nL/N$ takes the value

$$
\psi(x_n,t) = {1\over N}
              \sum_{k=1}^{N-1} b_k \sin \biggl( {\pi k n\over N} \biggr)\>
              \exp \biggl( i{\pi^2\hbar k^2\over2mL^2} t \biggr),
$$

where the $b_k$ are some set of (possibly complex) coefficients that
specify the exact shape of the wavefunction and the leading factor of $1/N$
is optional but convenient.

Since the Schrodinger equation (unlike the wave equation) is first order
in time, we need only a single initial condition on the value
of $\psi(x,t)$ to specify the coefficients $b_k$, although, since the
coefficients are in general complex, we will need to calculate both real
and imaginary parts of each coefficient.

As in Exercise 9.8 we consider an electron (mass
$M=9.109\times10^{-31}\,$ kg) in a box of length~$L=10^{-8}\,$ m.  At time
$t=0$ the wavefunction of the electron has the form

$$
\psi(x,0) = \exp \biggl[ -{(x-x_0)^2\over2\sigma^2} \biggr]
            e^{i\kappa x},
$$

where

$$
x_0 = {L\over2},\qquad
\sigma = 1\times10^{-10}\,\mathrm{m},\qquad
\kappa = 5\times10^{10}\,\mathrm{m}^{-1},
$$

and $\psi=0$ on the walls at $x=0$ and $x=L$.

Write a program to calculate the values of the coefficients $b_k$,
which for convenience can be broken down into their real and imaginary
parts as $b_k=\alpha_k+i\eta_k$.  Divide the box into $N=1000$ slices
and create two arrays containing the real and imaginary parts of
$\psi(x_n,0)$ at each grid point.  Perform discrete sine
transforms on each array separately and hence calculate the values of
the $\alpha_k$ and $\eta_k$ for all $k=1\ldots N-1$.

To perform the discrete sine transforms, you can use the fast transform
function dst from the package dcst, which you can find in
the online resources in the file named dcst.py.  A copy of the
code for the package can also be found in Appendix E.  The function takes
an array of $N$ real numbers and returns the discrete sine transform as
another array of $N$ numbers.

(Note that the first element of the input array should in principle
always be zero for a sine transform, but if it is not the dst
function will simply pretend that it is.  Similarly the first element of
the returned array is always zero, since the $k=0$ coefficient of a sine
transform is always zero.  So in effect, the sine transform really only
takes $N-1$ real numbers and transforms them into another $N-1$ real
numbers.  In some implementations of the discrete sine transform,
therefore, though not the one in the package dsct used here, the
first element of each array is simply omitted, since it's always zero
anyway, and the arrays are only $N-1$ elements long.)


Putting $b_k=\alpha_k+i\eta_k$ in the solution above and taking the real part we get

$$
\text{Re} \psi(x_n,t) = {1\over N} \sum_{k=1}^{N-1}
            \biggl[ \alpha_k \cos \biggl( {\pi^2\hbar k^2\over2mL^2} t \biggr)
            - \eta_k \sin \biggl( {\pi^2\hbar k^2\over2mL^2} t \biggr) \biggr]
            \sin \biggl( {\pi k n\over N} \biggr)
$$

for the real part of the wavefunction.  This is an inverse sine
transform with coefficients equal to the quantities in the square brackets.
Extend your program to calculate the real part of the
wavefunction $\psi(x,t)$ at an arbitrary time $t$ using this formula and
the inverse discrete sine transform function idst, also from the
package dcst.  Test your program by making a graph of the
wavefunction at time $t=10^{-16}\,$ s.